In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from trading.operations import create_trading_app, setup_trading_app, place_trade, cancel_trade, accept_trade, close_trading
from store.operations import *
from utils import *
from account import Account

In [2]:
dotenv.load_dotenv('.env')

ALGOD_URL = os.environ.get('ALGOD_URL')
ALGOD_TOKEN = os.environ.get('ALGOD_TOKEN')
print(f"ALGOD_URL: {ALGOD_URL}")
print(f"ALGOD_TOKEN: {ALGOD_TOKEN}")

client = get_algod_client(ALGOD_URL, ALGOD_TOKEN)

ALGOD_URL: http://localhost:4001
ALGOD_TOKEN: aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa


In [3]:
creator = Account.from_mnemonic(os.environ.get('CREATOR_MN'))
bidder = Account.from_mnemonic(os.environ.get('BUYER_MN'))
seller = Account.from_mnemonic(os.environ.get('SELLER_MN'))

staking_address = Account.from_mnemonic(os.environ.get("CREATOR_MN"))
team_wallet_address = Account.from_mnemonic(os.environ.get("TEAM_MN"))

storeAppID = int(os.environ.get("STORE_APP_ID"))
print(f"storeAppID: {storeAppID}")
#close_trading(client, 693, storeAppID, creator)

print(f"Creator address: {creator.get_address()}")
print(f"Bidder address: {bidder.get_address()}")
print(f"Seller Address: {seller.get_address()}")

storeAppID: 13
Creator address: 4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64
Bidder address: X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU
Seller Address: CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA


Alice is generating an example token...

In [5]:
asset_id = int(os.environ.get("TOKEN_ID"))

json_staring = "{\"platform\":\"ArtBlocksCurated\",\"tokenID\":\"2244\",\"series\":1,\"animation_url\":\"https://generator.artblocks.io/2244\",\"image\":\"https://media.artblocks.io/2244.png\",\"external_url\":\"https://artblocks.io/token/2244\",\"aspect_ratio\":1.5,\"payout_address\":\"0x6C093Fe8bc59e1e0cAe2Ec10F0B717D3D182056B\",\"interactive_nft\":{\"version\":\"0.0.9\",\"code_uri\":\"https://generator.artblocks.io/2244\"},\"name\":\"ChromieSquiggle#2244\",\"artist\":\"Snowfro\",\"description\":\"Simpleandeasilyidentifiable,eachsquiggleembodiesthesouloftheArtBlocksplatform.Considereachmypersonalsignatureasanartist,developer,andtinkerer.PublicmintingoftheChromieSquiggleispermanentlypaused.Theyarenowreservedformanualdistributiontocollectorsandcommunitymembersoveralongerperiodoftime.PleasevisitOpenSeatoexploreSquigglesavailableonthesecondarymarket.\",\"script_type\":\"p5js\",\"project_id\":\"0\",\"curation_status\":\"curated\",\"generator_url\":\"https://generator.artblocks.io/2244\",\"royaltyInfo\":{\"artistAddress\":\"0xb998a2520907ed1fc0f9f457b2219fb2720466cd\",\"additionalPayee\":\"0xf3860788d1597cecf938424baabe976fac87dc26\",\"additionalPayeePercentage\":\"100\",\"royaltyFeeByID\":\"5\"},\"collection_name\":\"ChromieSquigglebySnowfro\",\"website\":\"https://www.twitter.com/artonblockchain\",\"features\":{\"Type\":\"Normal\",\"Height\":\"3\",\"Segments\":16,\"Spectrum\":\"Normal\",\"EndColor\":197,\"StartColor\":87,\"ColorSpread\":\"29\",\"StepsBetween\":\"200\",\"ColorDirection\":\"Reverse\"},\"traits\":[{\"trait_type\":\"ChromieSquiggle\",\"value\":\"AllChromieSquiggles\"},{\"trait_type\":\"ChromieSquiggle\",\"value\":\"Type:Normal\"},{\"trait_type\":\"ChromieSquiggle\",\"value\":\"Height:3\"},{\"trait_type\":\"ChromieSquiggle\",\"value\":\"Segments:16\"},{\"trait_type\":\"ChromieSquiggle\",\"value\":\"Spectrum:Normal\"},{\"trait_type\":\"ChromieSquiggle\",\"value\":\"ColorSpread:29\"},{\"trait_type\":\"ChromieSquiggle\",\"value\":\"StepsBetween:200\"},{\"trait_type\":\"ChromieSquiggle\",\"value\":\"ColorDirection:Reverse\"}],\"token_hash\":\"0x7bf834ca6d71764aad5093323daad12a2afbde61d7a7cbf6db3191cb88572c3c\",\"is_static\":false,\"license\":\"NFTLicense\"}"
metaDataHash = hashMetaData(json_staring)

def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
    txn = transaction.AssetConfigTxn(
        sender=sender.get_address(),
        sp=client.suggested_params(),
        total=total,
        default_frozen=False,
        decimals=decimals,
        unit_name=unit_name,
        asset_name=asset_name,
        manager=creator.get_address(),
        reserve=creator.get_address(),
        freeze=creator.get_address(),
        clawback=creator.get_address(),
        url="https://token.artblocks.io/2244",
        metadata_hash=metaDataHash,
        strict_empty_address_check=False,
    )
    signed_txn = txn.sign(sender.get_private_key())

    client.send_transaction(signed_txn)

    response = wait_for_confirmation(client, signed_txn.get_txid())
    assert response.asset_index is not None and response.asset_index > 0
    return response.asset_index

asset_id = create_dummy_asset(seller, 100000000000000000, 0, "my token", "MNFT")

print(f"The token id is: {asset_id}")

Asset metadata in base64: 
Dfbj7kL4iQx9jkm800+MX5txns3Htka/xp9tgo8p4w4=
Waiting for confirmation...
Transaction VTQETFGCQOQKL453AKMVZ6PCBAMDQC5S7TDMEDDOZ6IATEXPRU3A confirmed in round 75745.
The token id is: 397


In [8]:
start_time = int(time()) + 10
end_time = start_time + 10
price = 1_000_000  # 1 Algo

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [5]:
deleteApps(client, [12], creator)

AlgodHTTPError: TransactionPool.Remember: transaction KTJVLW5HTMCEE2GFAMFI5KWVE6IXKRPZBS5G7MZMUT2KGWWA645Q: only clearing out is supported for applications that do not exist

In [4]:
storeAppID = int(os.environ.get("STORE_APP_ID"))

#storeAppID = create_store_app(client, creator)
print(f"Store App ID: {storeAppID}")

storeAppAddress = get_application_address(storeAppID)
print(f"Store App Address: {storeAppAddress}")


Store App ID: 13
Store App Address: WI7XTCTK42LLU6XD5AWB6L5Q2JF3F2SYI2HMMQD65YMQAATOB75KGAE7PQ


In [7]:

appID = 5012
appID = create_trading_app(
    client=client,
    creator=creator,
    store_app_id=storeAppID,
    staking_address=staking_address.get_address(),
    team_wallet_address=team_wallet_address.get_address()
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction CCUPDBYYG7Q6I7XDVBYNQG3CHZCG7Q3ALNVNW2ARC2KOC5LWX4XQ confirmed in round 29700.
Waiting for confirmation...
Transaction 25WDVFIKJAK5NUNHQFQUCTMPBHGRGIOPAROPIE46WDVVLPMVZBWQ confirmed in round 29702.
App ID: 147
App Address: GBOCGMWXDET4ZN6YAZBMEP3RCSNQRPHZHV7SQ3DGJRDRXUK35XFEHTHXWA


In [ ]:
#setup store app
set_up(client=client, creator=creator, app_id=storeAppID, trade_app_id=appID, bid_app_id=appID, auction_app_id=appID, distribution_app_id=appID)

Alice is setting up and funding token auction...

In [8]:
setup_trading_app(
    client=client,
    app_id=appID,
    funder=creator,
    token_id=asset_id,
)

sellerAlgosBefore = get_balances(client, seller.get_address())[0]
print("Alice's algo balance: ", sellerAlgosBefore, " algos")

Waiting for confirmation...
Transaction 2P665LHFGYMCIWIB5Q7GYGUW276NDGX6IE25PEMYYMGGXQYWOVQA confirmed in round 29719.
Alice's algo balance:  10007389150592  algos


In [13]:
appBalance = get_balances(client, get_application_address(appID))[0]
print("Alice's algo balance: ", appBalance, " algos")

Alice's algo balance:  201000  algos


In [14]:
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 201000, 3558: 0}


In [ ]:
bidPrice = price
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidPrice, " algos")

In [ ]:
trading_index = place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=10, price=bidPrice, trading_index="")

print("Carla bid", trading_index)

In [ ]:
# account_info = client.account_info(bidder.get_address())  
# print(account_info.get('apps-local-state', []))
# print(is_opted_in_app(client, appID, bidder.get_address()))

cancel_trade(client, appID, seller, trading_index)
print("Carla cancel trading", trading_index)

In [ ]:
trading_index = place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=10, price=bidPrice, trading_index="")

place_trade(client=client, app_id=appID, seller=seller, token_id=asset_id, token_amount=15, price=bidPrice, trading_index=trading_index)

In [ ]:
accept_trade(client, appID, bidder, seller.get_address(), trading_index)

Alice is closing out the auction....

In [ ]:
sleep(5)

close_trading(client, appID, creator, [asset_id])

In [ ]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

#assert actualAppBalances == expectedAppBalances

In [ ]:
bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]
print("Carla's NFT balance:", bidderNftBalance, " for token ID: ", asset_id)

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")

actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee

In [15]:
appID = 5024
app_global_state = get_app_global_state(client, appID)
print(app_global_state)
seller_app_local_state = get_app_local_state(client, storeAppID, seller.get_address())
print(seller_app_local_state)
bidder_app_local_state = get_app_local_state(client, storeAppID, bidder.get_address())
print(bidder_app_local_state)
# print(encoding.encode_address(app_global_state[b"AS"]))
# print(encoding.encode_address(app_global_state[b"S"]))
#print(encoding.encode_address(storeAppState[b"C"]))

{b'TBA': 0, b'TSA': 0}
{}
{}
